In [1]:
intents_data = {
    "intents": [
        {
            "tag": "greeting",
            "patterns": ["Hi", "Hey", "Is anyone there?", "Hello", "Hay"],
            "responses": ["Hello", "Hi", "Hi there"]
        },
        {
            "tag": "goodbye",
            "patterns": ["Bye", "See you later", "Goodbye"],
            "responses": ["See you later", "Have a nice day", "Bye! Come back again"]
        },
        {
            "tag": "thanks",
            "patterns": ["Thanks", "Thank you", "That's helpful", "Thanks for the help"],
            "responses": ["Happy to help!", "Any time!", "My pleasure", "You're most welcome!"]
        },
        {
            "tag": "about",
            "patterns": ["Who are you?", "What are you?", "Who you are?"],
            "responses": ["I.m Joana, your bot assistant", "I'm Joana, an Artificial Intelligent bot"]
        },
        {
            "tag": "name",
            "patterns": ["what is your name", "what should I call you", "whats your name?"],
            "responses": ["You can call me Joana.", "I'm Joana!", "Just call me as Joana"]
        },
        {
            "tag": "help",
            "patterns": ["Could you help me?", "give me a hand please", "Can you help?", "What can you do for me?", "I need a support", "I need a help", "support me please"],
            "responses": ["Tell me how can assist you", "Tell me your problem to assist you", "Yes Sure, How can I support you"]
        },
        {
            "tag": "createaccount",
            "patterns": ["I need to create a new account", "how to open a new account", "I want to create an account", "can you create an account for me", "how to open a new account"],
            "responses": ["You can just easily create a new account from our web site", "Just go to our web site and follow the guidelines to create a new account"]
        },
        {
            "tag": "complaint",
            "patterns": ["have a complaint", "I want to raise a complaint", "there is a complaint about a service"],
            "responses": ["Please provide us your complaint in order to assist you", "Please mention your complaint, we will reach you and sorry for any inconvenience caused"]
        }
    ]
}


In [2]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [3]:
training_sentences = []
training_labels = []
labels = []
responses = []

for intent in intents_data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])

num_classes = len(labels)

In [4]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [5]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts([pattern for intent in intents_data['intents'] for pattern in intent['patterns']])
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences([pattern for intent in intents_data['intents'] for pattern in intent['patterns']])
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs) 
for layer in model.layers:
    if isinstance(layer, Embedding):
        total_params = layer.count_params()
        output_shape = (None, max_len, embedding_dim)
    elif isinstance(layer, GlobalAveragePooling1D):
        total_params = layer.count_params()
        output_shape = (None, embedding_dim)
    elif isinstance(layer, Dense):
        total_params = layer.count_params()
        output_shape = (None, 16) if layer.activation == 'relu' else (None, num_classes)
    else:
        total_params = 0
        output_shape = None
    print(f"Layer: {layer.name}, Total params: {total_params}, Output shape: {output_shape}")

# Assuming batch size of 1 for simplicity
batch_size = 1
sample_input = np.ones((batch_size, max_len))
output_shape = model.predict(sample_input).shape
print(f"Model output shape: {output_shape}")

C:\Users\Indi Gammanpila\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ ?                      │   0 (unbuilt) │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

...

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4182 - loss: 1.7078 
Epoch 206/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4182 - loss: 1.7015 
Epoch 207/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4899 - loss: 1.6868 
Epoch 208/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5205 - loss: 1.6830 
Epoch 209/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5205 - loss: 1.6808 
Epoch 210/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4899 - loss: 1.6796 
Epoch 211/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3876 - loss: 1.6896 
Epoch 212/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3876 - loss: 1.6879 
Epoch 213/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4182 - loss: 1.6840 
Epoch 214/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5205 - loss: 1.6704 
Epoch 215/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5101 - loss: 1.6767 
Epoch 216/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.520

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5101 - loss: 1.5333 
Epoch 274/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5205 - loss: 1.5219 
Epoch 275/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5101 - loss: 1.5314 
Epoch 276/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5205 - loss: 1.5230 
Epoch 277/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5101 - loss: 1.5264 
Epoch 278/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5101 - loss: 1.5253 
Epoch 279/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5205 - loss: 1.5204 
Epoch 280/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4899 - loss: 1.5189 
Epoch 281/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4795 - loss: 1.5370 
Epoch 282/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4795 - loss: 1.5216 
Epoch 283/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4795 - loss: 1.5140 
Epoch 284/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.489

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5101 - loss: 1.3873 
Epoch 342/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5101 - loss: 1.3826 
Epoch 343/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5205 - loss: 1.3634 
Epoch 344/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5205 - loss: 1.3633 
Epoch 345/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5205 - loss: 1.3609 
Epoch 346/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5205 - loss: 1.3568 
Epoch 347/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5101 - loss: 1.3699 
Epoch 348/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5101 - loss: 1.3757 
Epoch 349/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4489 - loss: 1.3779 
Epoch 350/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4489 - loss: 1.3709 
Epoch 351/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4489 - loss: 1.3588 
Epoch 352/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.459

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5713 - loss: 1.2136 
Epoch 410/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5713 - loss: 1.2112 
Epoch 411/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5407 - loss: 1.2161 
Epoch 412/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5713 - loss: 1.1984 
Epoch 413/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5818 - loss: 1.1858 
Epoch 414/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5713 - loss: 1.1928 
Epoch 415/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5818 - loss: 1.1888 
Epoch 416/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5818 - loss: 1.1942 
Epoch 417/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5205 - loss: 1.1944 
Epoch 418/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4899 - loss: 1.2073 
Epoch 419/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5205 - loss: 1.2045 
Epoch 420/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.581

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6326 - loss: 1.0236 
Epoch 478/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6326 - loss: 1.0270 
Epoch 479/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6124 - loss: 0.9989 
Epoch 480/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6020 - loss: 0.9979 
Epoch 481/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6632 - loss: 1.0011 
Epoch 482/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6632 - loss: 0.9949 
Epoch 483/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6736 - loss: 0.9867 
Epoch 484/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6736 - loss: 0.9834 
Epoch 485/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6632 - loss: 0.9955 
Epoch 486/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6632 - loss: 1.0083 
Epoch 487/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6632 - loss: 0.9955 
Epoch 488/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.673

In [6]:
import pickle

# Save the trained model
model.export("chatmodel")

# Save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)


INFO:tensorflow:Assets written to: chatmodel\assets


INFO:tensorflow:Assets written to: chatmodel\assets


Saved artifact at 'chatmodel'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 20), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 8), dtype=tf.float32, name=None)
Captures:
  1775685487072: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1775685486720: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1775685487600: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1775685487776: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1775685487952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1775685489008: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1775685488656: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [11]:
import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import random
import pickle

intents_data = {
    "intents": [
        {
            "tag": "greeting",
            "patterns": ["Hi", "Hey", "Is anyone there?", "Hello", "Hay"],
            "responses": ["Hello", "Hi", "Hi there"]
        },
        {
            "tag": "goodbye",
            "patterns": ["Bye", "See you later", "Goodbye"],
            "responses": ["See you later", "Have a nice day", "Bye! Come back again"]
        },
        {
            "tag": "thanks",
            "patterns": ["Thanks", "Thank you", "That's helpful", "Thanks for the help"],
            "responses": ["Happy to help!", "Any time!", "My pleasure", "You're most welcome!"]
        },
        {
            "tag": "about",
            "patterns": ["Who are you?", "What are you?", "Who you are?"],
            "responses": ["I.m Joana, your bot assistant", "I'm Joana, an Artificial Intelligent bot"]
        },
        {
            "tag": "name",
            "patterns": ["what is your name", "what should I call you", "whats your name?"],
            "responses": ["You can call me Joana.", "I'm Joana!", "Just call me as Joana"]
        },
        {
            "tag": "help",
            "patterns": ["Could you help me?", "give me a hand please", "Can you help?", "What can you do for me?", "I need a support", "I need a help", "support me please"],
            "responses": ["Tell me how can assist you", "Tell me your problem to assist you", "Yes Sure, How can I support you"]
        },
        {
            "tag": "createaccount",
            "patterns": ["I need to create a new account", "how to open a new account", "I want to create an account", "can you create an account for me", "how to open a new account"],
            "responses": ["You can just easily create a new account from our web site", "Just go to our web site and follow the guidelines to create a new account"]
        },
        {
            "tag": "complaint",
            "patterns": ["have a complaint", "I want to raise a complaint", "there is a complaint about a service"],
            "responses": ["Please provide us your complaint in order to assist you", "Please mention your complaint, we will reach you and sorry for any inconvenience caused"]
        }
    ]
}

def chat():
    # load trained model
    model = keras.layers.TFSMLayer('chatmodel', call_endpoint='serving_default')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20
    
    while True:
        print("User: " , end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in intents_data['intents']:
            if i['tag'] == tag:
                print("ChatBot:" , np.random.choice(i['responses']))

print("Start messaging with the bot (type quit to stop)!" )
chat()

Start messaging with the bot (type quit to stop)!
User: quit
